<p>
  <a href="https://colab.research.google.com/github/neo4j-partners/hands-on-lab-neo4j-and-vertex-ai/blob/main/Lab%204%20-%20Exploring%20Data/exploring_cypher.ipynb" target="_blank">
    <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
  </a>
</p>

First off, you'll also need to install a few packages.

In [ ]:
!pip install --quiet --upgrade neo4j
!pip install --quiet google-cloud-storage

You'll need to enter the credentials from your Neo4j instance below.

The default DB_NAME is always neo4j.

In [ ]:
DB_URL = "neo4j://34.138.21.128:7687"
DB_USER = "neo4j"
DB_PASS = "foo123"
DB_NAME = "neo4j"

In [ ]:
import pandas as pd
from neo4j import GraphDatabase

driver = GraphDatabase.driver(DB_URL, auth=(DB_USER, DB_PASS))

In [ ]:
# node labels
with driver.session(database=DB_NAME) as session:
  result = session.read_transaction(
    lambda tx: tx.run(
      """
        CALL db.labels() YIELD label
        CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as freq', {})
        YIELD value
        RETURN label, value.freq AS freq
      """
    ).data()
  )
df = pd.DataFrame(result)
display(df)

In [ ]:
# relationship types
with driver.session(database=DB_NAME) as session:
    result = session.read_transaction(
        lambda tx: tx.run(
            """
      CALL db.relationshipTypes() YIELD relationshipType as type
      CALL apoc.cypher.run('MATCH ()-[:`'+type+'`]->() RETURN count(*) as freq', {})
      YIELD value
      RETURN type AS relationshipType, value.freq AS freq
      ORDER by freq DESC
      """
        ).data()
    )
df = pd.DataFrame(result)
display(df)